In [ ]:
import sys
import os
sys.path.append('../')
import pathlib as pl
from SymEigen import *
from sympy import symbols
from project_dir import project_dir
Gen = EigenFunctionGenerator()
Gen.MacroBeforeFunction("__host__ __device__")


In [ ]:
x = Eigen.Vector("x", 3)
P = Eigen.Vector("P", 3)
N = Eigen.Vector("N", 3)

Cl0 = Gen.Closure(x, P, N)

D = ((x-P).dot(N))**2
D

In [ ]:
dDdx = VecDiff(D, x)
dDdx

In [ ]:
ddDddx = VecDiff(dDdx, x)
ddDddx

In [ ]:
x0 = Eigen.Vector("x_prev", 3)
e1 = Eigen.Vector("e1", 3)
e2 = Eigen.Vector("e2", 3)

dx = x - x0
dx

In [ ]:
u = dx.dot(e1)
v = dx.dot(e2)

TR = Matrix([[u], [v]])
TR

In [ ]:
dTRdx = VecDiff(TR, x)
dTRdx

In [ ]:
Cl1 = Gen.Closure(x, x0, e1, e2)

In [ ]:
s = f'''
// > Squared Version
// > D := d*d 
// > DHat := dHat*dHat

{Cl0("HalfPlaneD",D)}
{Cl0("dHalfPlaneDdx",dDdx)}
{Cl0("ddHalfPlaneDddx",ddDddx)}
{Cl1("TR",TR)}
{Cl1("dTRdx",dTRdx)}
'''
print(s)

f = open( backend_source_dir('cuda') / 'contact_system/contact_models/sym/vertex_half_plane_distance.inl', 'w')
f.write(s)
f.close()